<a href="https://colab.research.google.com/github/brunocosouza/supervised_learning/blob/main/MO432_Atividade_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MO432 - Atividade 4

RA234837 - Bruno César de Oliveira Souza

RA065068 - Walker Humberto Batista

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

import matplotlib
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

Attribute Information:

There are 6 numerical and 8 categorical attributes. The labels have been changed for the convenience of the statistical algorithms. For example, attribute 4 originally had 3 labels p,g,gg and these have been changed to labels 1,2,3.

1. A1: 0,1 CATEGORICAL (formerly: a,b)
2. A2: continuous.
3. A3: continuous.
4. A4: 1,2,3 CATEGORICAL (formerly: p,g,gg)
5. A5: 1, 2,3,4,5, 6,7,8,9,10,11,12,13,14 CATEGORICAL (formerly: ff,d,i,k,j,aa m,c,w, e, q, r,cc, x)
6. A6: 1, 2,3, 4,5,6,7,8,9 CATEGORICAL (formerly: ff,dd,j,bb,v,n,o,h,z)
7. A7: continuous.
8. A8: 1, 0 CATEGORICAL (formerly: t, f)
9. A9: 1, 0 CATEGORICAL (formerly: t, f)
10. A10: continuous.
11. A11: 1, 0 CATEGORICAL (formerly t, f)
12. A12: 1, 2, 3 CATEGORICAL (formerly: s, g, p)
13. A13: continuous.
14. A14: continuous.
15. A15: 1,2 class attribute (formerly: +,-)

In [ ]:
# Reading data into pandas dataframe
df_raw = pd.read_csv('https://www.ic.unicamp.br/~wainer/cursos/1s2021/432/dados4.csv')

In [ ]:
df_X = df_raw[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14']]
df_Y = df_raw[['V15']]

In [ ]:
df_X = pd.get_dummies(df_X)
df_X.head()

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,V14,V5_aa,V5_c,V5_cc,V5_d,V5_e,V5_ff,V5_i,V5_j,V5_k,V5_m,V5_q,V5_r,V5_w,V5_x,V6_bb,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
0,1,22.08,11.46,2,1.585,0,0,0,1,100,1213,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1,0,22.67,7.00,2,0.165,0,0,0,0,160,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,0,29.58,1.75,1,1.250,0,0,0,1,280,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
3,0,21.67,11.50,1,0.000,1,1,11,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
4,1,20.17,8.17,2,1.960,1,1,14,0,60,159,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0


## Centralizando e Normalizando atributos

Transformanmos nossos dados em matrizes do numpy para utilizar as funções do sklearn para centralizarmos as amostras e normalizarmos em relação ao desvio padrão.

In [ ]:
# Numpy transformation
x = df_X.to_numpy()
y = df_Y.to_numpy()
x.shape, y.shape

((690, 36), (690, 1))

In [ ]:
# Centering and scaling convertion
scaler = StandardScaler()
x_hat = scaler.fit_transform(x)

y_hat = y

# Funções auxiliares

Usaremos Nested Cross Validation para encontrar o melhor modelo com os melhores hiperparametros. A ideia é separar a seleção de hiperparametros da seleção de algoritmos de classificação. Assim queremos saber qual é o melhor algoritmo à ser usado num conjunto de dado sem escolher o melhor conjunto de hiperparametros para esse algoritmo ao mesmo tempo.

Para esse método, criamos uma função principal que executa o experimento e duas função correspondentes ao Nested Cross Validation : outerloop e innerloop.

## Métrica

Avaliamos os algoritmos com os melhores hiperparametros usando a métrica ROC AUC.

A Curva Característica de Operação do Receptor, ou, do inglês, Receiver Operating Characteristic Curve, ou, simplesmente, curva ROC, é uma representação gráfica que ilustra o desempenho de um sistema classificador binário à medida que o seu limiar de discriminação varia.

In [ ]:
def experiment(x, y, estimators):
  results = []

  for (estimator, search_params, estimator_name) in estimators:
    result = outerloop(x, y, estimator, search_params, estimator_name)
    results.append(result)

  return results

def outerloop(x, y, estimator, search_parameters, estimator_name):
  result = {}
  results = []

  for i in range(4):
    x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                        test_size=0.3, 
                                                        random_state=1024 + i)
    if search_parameters is None:
      (best_estimator, best_params) = (estimator, None)
    else:
      (best_estimator, best_params) = innerloop(x_train, y_train, estimator, 
                                                search_parameters)

    best_estimator.fit(x_train, y_train)
    y_estimated = best_estimator.predict(x_test)

    auc = roc_auc_score(y_test, y_estimated)

    results.append(auc)
  
  #result['hyperparameters'] = best_params
  result['name'] = estimator_name
  result['mean_auc'] = np.mean(results)

  return result

def innerloop(x, y, estimator, search_parameters):
  cv = StratifiedKFold(n_splits=3, 
                       shuffle=False,
                       random_state=1024)
  gs = RandomizedSearchCV(n_iter=10,
                          estimator=estimator, 
                          param_distributions=search_parameters, 
                          cv=cv, 
                          scoring="roc_auc", 
                          n_jobs=-1)
  gs.fit(x,y)

  return gs.best_estimator_, gs.best_params_

# Estimators

1. Logistic regression sem regularização
2. Logistic regression com regularização L2
3. LDA
4. QDA
5. SVM Linear
6. SVM com kernel RBF
7. Naive Bayes
8. KNN
9. MLP
10. Arvore de decisão
11. Random Forest
12. GBM

In [ ]:
estimators = [(LogisticRegression(penalty='none'), None, 
               'Logistic regression sem regularização'), 
              (LogisticRegression(penalty='l2'), 
               {'C': 1 / np.power(10, np.random.uniform(low=-3, high=3, size=(10,)))},
               'Logistic regression com regularização L2'),
              (LinearDiscriminantAnalysis(), None, 
               'LDA'),
              (QuadraticDiscriminantAnalysis(), None, 
               'QDA'),
              (SVC(kernel='linear', max_iter=1000), 
               {'C': np.power(10, np.random.uniform(low=-5, high=15, size=(10,)))},
               'SVM Linear'),
              (SVC(kernel='rbf', max_iter=1000), 
               {'C': np.power(10, np.random.uniform(low=-5, high=15, size=(10,))),
                'gamma': np.power(10, np.random.uniform(low=-9, high=3, size=(10,)))},
               'SVM com kernel RBF'),
              (GaussianNB(), None,
               'Naive Bayes'),
              (KNeighborsClassifier(), 
                {'n_neighbors': np.random.randint(low=1, high=301, size=(10,))},
               'KNN'),
              (MLPClassifier(), 
               {'hidden_layer_sizes': np.random.randint(low=5, high=20, size=(10,))},
               'MLP'),
              (DecisionTreeClassifier(), 
               {'ccp_alpha': np.random.uniform(low=0, high=0.04, size=(10,))},
               'Árvore de decisão'),
              (RandomForestClassifier(), 
               {'n_estimators': [10, 100, 1000],
                'max_features': [5, 8, 10]},
               'Random Forest'),
              (GradientBoostingClassifier(), 
               {'n_estimators': np.random.randint(low=5, high=100, size=(10,)),
                'learning_rate': np.random.uniform(low=0.01, high=0.3, size=(10,)),
                'max_depth': [2, 3]},
               'GBM')]

# Experimento

Aqui rodamos o codigo principal do experimento

In [ ]:
exp_results = experiment(x_hat, y_hat, estimators)

# Resultados

O melhor algoritmo foi o estimador Random Forest de acordo com a métrica AUC. Segue a tabela de resultados por algoritmo testado.

In [ ]:
df = pd.DataFrame(exp_results)
df.columns = ['Método', 'AUC Médio']
df.set_index(['Método'], inplace=True)

print(df)

                                          AUC Médio
Método                                             
Logistic regression sem regularização      0.850834
Logistic regression com regularização L2   0.869028
LDA                                        0.862771
QDA                                        0.725328
SVM Linear                                 0.847478
SVM com kernel RBF                         0.586056
Naive Bayes                                0.739205
KNN                                        0.789448
MLP                                        0.861369
Árvore de decisão                          0.862422
Random Forest                              0.873468
GBM                                        0.870842
